In [3]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "collapsed": true,
    "pycharm": {
     "name": "#%% md\n"
    }
   },
   "source": [
    "#### import packages, create connection to snowflake"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "outputs": [],
   "source": [
    "# import packages\n",
    "from pyLCA import connect_to_snowflake, upload_to_snowflake\n",
    "import pandas as pd\n",
    "import os\n",
    "import os\n",
    "import getpass\n",
    "import pandas as pd\n",
    "import tempfile\n",
    "import numpy as np\n",
    "\n",
    "# create connection to snowflake\n",
    "# IF RUNNING LOCALLY CHANGE ENVIRONMENT TO 'local'\n",
    "sf_engine = connect_to_snowflake(user=os.environ.get('snowflake_uid'), password=os.environ.get('snowflake_pw'),\n",
    "                                 schema='TD_REPORTING', environment='local')\n",
    "sf_con = sf_engine.connect()"
   ],
    
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%%\n"
    }
   }
  },
  {
   "cell_type": "markdown",
   "source": [
    "#### define function to get distance between 2 points"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%% md\n"
    }
   }
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "outputs": [],
   "source": [
    "def haversine_np(lon1, lat1, lon2, lat2):\n",
    "    \"\"\"\n",
    "    Calculate the great circle distance between two points\n",
    "    on the earth (specified in decimal degrees)\n",
    "\n",
    "    All args must be of equal length.\n",
    "\n",
    "    \"\"\"\n",
    "    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])\n",
    "\n",
    "    dlon = lon2 - lon1\n",
    "    dlat = lat2 - lat1\n",
    "\n",
    "    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2\n",
    "\n",
    "    c = 2 * np.arcsin(np.sqrt(a))\n",
    "    km = 6367 * c\n",
    "    return km"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%%\n"
    }
   }
  },
  {
   "cell_type": "markdown",
   "source": [
    "#### get location of petrol stores"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%% md\n"
    }
   }
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "outputs": [],
   "source": [
    "query = \"\"\"select store_cd,\n",
    "                   store_name,\n",
    "                   store_type_finance_desc,\n",
    "                   postcode as postcode_store,\n",
    "                   latitude as latitude_store,\n",
    "                   longitude as longitude_store\n",
    "                   from ADW_PROD.ADW_PROPERTY_PL.DIM_STORE\n",
    "                   where js_petrol_flag = 'Y'\"\"\"\n",
    "petrol_store_data = pd.read_sql(query, sf_con)\n",
    "petrol_store_data = petrol_store_data.dropna()"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%%\n"
    }
   }
  },
  {
   "cell_type": "markdown",
   "source": [
    "#### get customer locations"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%% md\n"
    }
   }
  },
  {
   "cell_type": "code",
   "execution_count": 38,
   "outputs": [
    {
     "data": {
      "text/plain": "         postcode   latitude  longitude\n0        AB10 1XG  57.144156  -2.114864\n1        AB10 6RN  57.137871  -2.121487\n2        AB10 7JB  57.124274  -2.127206\n3        AB11 5QN  57.142701  -2.093295\n4        AB11 6UL  57.137468  -2.112455\n...           ...        ...        ...\n1785249   YO8 9TX  53.771523  -1.120955\n1785250   YO8 9UE  53.771428  -1.160421\n1785251   YO8 9UT  53.763194  -1.073526\n1785252   ZE2 9FT  60.000249  -1.237057\n1785253   ZE2 9FW  60.041164  -1.208829\n\n[1772720 rows x 3 columns]",
      "text/html": "<div>\n<style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n</style>\n<table border=\"1\" class=\"dataframe\">\n  <thead>\n    <tr style=\"text-align: right;\">\n      <th></th>\n      <th>postcode</th>\n      <th>latitude</th>\n      <th>longitude</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>AB10 1XG</td>\n      <td>57.144156</td>\n      <td>-2.114864</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>AB10 6RN</td>\n      <td>57.137871</td>\n      <td>-2.121487</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>AB10 7JB</td>\n      <td>57.124274</td>\n      <td>-2.127206</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>AB11 5QN</td>\n      <td>57.142701</td>\n      <td>-2.093295</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>AB11 6UL</td>\n      <td>57.137468</td>\n      <td>-2.112455</td>\n    </tr>\n    <tr>\n      <th>...</th>\n      <td>...</td>\n      <td>...</td>\n      <td>...</td>\n    </tr>\n    <tr>\n      <th>1785249</th>\n      <td>YO8 9TX</td>\n      <td>53.771523</td>\n      <td>-1.120955</td>\n    </tr>\n    <tr>\n      <th>1785250</th>\n      <td>YO8 9UE</td>\n      <td>53.771428</td>\n      <td>-1.160421</td>\n    </tr>\n    <tr>\n      <th>1785251</th>\n      <td>YO8 9UT</td>\n      <td>53.763194</td>\n      <td>-1.073526</td>\n    </tr>\n    <tr>\n      <th>1785252</th>\n      <td>ZE2 9FT</td>\n      <td>60.000249</td>\n      <td>-1.237057</td>\n    </tr>\n    <tr>\n      <th>1785253</th>\n      <td>ZE2 9FW</td>\n      <td>60.041164</td>\n      <td>-1.208829</td>\n    </tr>\n  </tbody>\n</table>\n<p>1772720 rows × 3 columns</p>\n</div>"
     },
     "execution_count": 38,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "csv_name = 'ukpostcodes.csv' # update the file name\n",
    "location = os.environ.get('Onedrive') + '\\Campaign Analytics Document Library\\Trade Driving\\Trade Driving Experimental\\Selections\\\\' + csv_name\n",
    "\n",
    "new_table_name = 'uk_postcodes_long_lat'\n",
    "postcode_data = pd.read_csv(location)\n",
    "postcode_data = postcode_data.dropna()\n",
    "postcode_data.drop('id', axis = 1)\n"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%%\n"
    }
   }
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "outputs": [],
   "source": [
    "postcode_data_avg = pd.read_sql(\"\"\"select postal_area2,\n",
    "                                          avg(latitude) as latitude,\n",
    "                                          avg(longitude) as longitude\n",
    "                                    from uk_postcodes_long_lat_area\n",
    "                                    group by 1\n",
    "                                \"\"\", sf_con)"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%%\n"
    }
   }
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "outputs": [
    {
     "data": {
      "text/plain": "  postal_area2   latitude  longitude\n0        CV6 5  52.427811  -1.497151\n1       CV10 7  52.510772  -1.485819\n2        CT6 8  51.367633   1.106363\n3        CR5 3  51.309163  -0.158019\n4        CR4 9  51.405130  -0.174797",
      "text/html": "<div>\n<style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n</style>\n<table border=\"1\" class=\"dataframe\">\n  <thead>\n    <tr style=\"text-align: right;\">\n      <th></th>\n      <th>postal_area2</th>\n      <th>latitude</th>\n      <th>longitude</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>CV6 5</td>\n      <td>52.427811</td>\n      <td>-1.497151</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>CV10 7</td>\n      <td>52.510772</td>\n      <td>-1.485819</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>CT6 8</td>\n      <td>51.367633</td>\n      <td>1.106363</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>CR5 3</td>\n      <td>51.309163</td>\n      <td>-0.158019</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>CR4 9</td>\n      <td>51.405130</td>\n      <td>-0.174797</td>\n    </tr>\n  </tbody>\n</table>\n</div>"
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "postcode_data_avg.head()"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%%\n"
    }
   }
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "outputs": [],
   "source": [
    "petrol_store_data['key'] = 1\n",
    "postcode_data_avg['key'] = 1\n",
    "final_data = pd.merge(petrol_store_data, postcode_data_avg, on='key').drop(\"key\", 1)\n"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%%\n"
    }
   }
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "outputs": [
    {
     "data": {
      "text/plain": "  store_cd           store_name store_type_finance_desc postcode_store  \\\n0     1420  Bishops Waltham PFS                     PFS       SO32 1BA   \n1     1420  Bishops Waltham PFS                     PFS       SO32 1BA   \n2     1420  Bishops Waltham PFS                     PFS       SO32 1BA   \n3     1420  Bishops Waltham PFS                     PFS       SO32 1BA   \n4     1420  Bishops Waltham PFS                     PFS       SO32 1BA   \n\n   latitude_store  longitude_store postal_area2   latitude  longitude  \n0         50.9541           -1.215        CV6 5  52.427811  -1.497151  \n1         50.9541           -1.215       CV10 7  52.510772  -1.485819  \n2         50.9541           -1.215        CT6 8  51.367633   1.106363  \n3         50.9541           -1.215        CR5 3  51.309163  -0.158019  \n4         50.9541           -1.215        CR4 9  51.405130  -0.174797  ",
      "text/html": "<div>\n<style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n</style>\n<table border=\"1\" class=\"dataframe\">\n  <thead>\n    <tr style=\"text-align: right;\">\n      <th></th>\n      <th>store_cd</th>\n      <th>store_name</th>\n      <th>store_type_finance_desc</th>\n      <th>postcode_store</th>\n      <th>latitude_store</th>\n      <th>longitude_store</th>\n      <th>postal_area2</th>\n      <th>latitude</th>\n      <th>longitude</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>1420</td>\n      <td>Bishops Waltham PFS</td>\n      <td>PFS</td>\n      <td>SO32 1BA</td>\n      <td>50.9541</td>\n      <td>-1.215</td>\n      <td>CV6 5</td>\n      <td>52.427811</td>\n      <td>-1.497151</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>1420</td>\n      <td>Bishops Waltham PFS</td>\n      <td>PFS</td>\n      <td>SO32 1BA</td>\n      <td>50.9541</td>\n      <td>-1.215</td>\n      <td>CV10 7</td>\n      <td>52.510772</td>\n      <td>-1.485819</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>1420</td>\n      <td>Bishops Waltham PFS</td>\n      <td>PFS</td>\n      <td>SO32 1BA</td>\n      <td>50.9541</td>\n      <td>-1.215</td>\n      <td>CT6 8</td>\n      <td>51.367633</td>\n      <td>1.106363</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>1420</td>\n      <td>Bishops Waltham PFS</td>\n      <td>PFS</td>\n      <td>SO32 1BA</td>\n      <td>50.9541</td>\n      <td>-1.215</td>\n      <td>CR5 3</td>\n      <td>51.309163</td>\n      <td>-0.158019</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>1420</td>\n      <td>Bishops Waltham PFS</td>\n      <td>PFS</td>\n      <td>SO32 1BA</td>\n      <td>50.9541</td>\n      <td>-1.215</td>\n      <td>CR4 9</td>\n      <td>51.405130</td>\n      <td>-0.174797</td>\n    </tr>\n  </tbody>\n</table>\n</div>"
     },
     "execution_count": 9,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "final_data.head()"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%%\n"
    }
   }
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "outputs": [
    {
     "data": {
      "text/plain": "      store_cd           store_name store_type_finance_desc postcode_store  \\\n1215      1420  Bishops Waltham PFS                     PFS       SO32 1BA   \n3916      1420  Bishops Waltham PFS                     PFS       SO32 1BA   \n10097     1420  Bishops Waltham PFS                     PFS       SO32 1BA   \n11912     1399  Aylesbury Gh Rd PFS                     PFS       HP19 8PQ   \n12623     1399  Aylesbury Gh Rd PFS                     PFS       HP19 8PQ   \n\n       latitude_store  longitude_store postal_area2   latitude  longitude  \\\n1215         50.95410         -1.21500       PO17 5  50.891426  -1.194287   \n3916         50.95410         -1.21500       SO32 2  50.934787  -1.217127   \n10097        50.95410         -1.21500       SO32 1  50.960326  -1.217575   \n11912        51.82202         -0.82514       HP27 7  51.778221  -0.771607   \n12623        51.82202         -0.82514       HP20 9  51.820160  -0.816227   \n\n       distance  \n1215   7.333661  \n3916   2.158665  \n10097  0.748518  \n11912  7.685940  \n12623  1.011785  ",
      "text/html": "<div>\n<style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n</style>\n<table border=\"1\" class=\"dataframe\">\n  <thead>\n    <tr style=\"text-align: right;\">\n      <th></th>\n      <th>store_cd</th>\n      <th>store_name</th>\n      <th>store_type_finance_desc</th>\n      <th>postcode_store</th>\n      <th>latitude_store</th>\n      <th>longitude_store</th>\n      <th>postal_area2</th>\n      <th>latitude</th>\n      <th>longitude</th>\n      <th>distance</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>1215</th>\n      <td>1420</td>\n      <td>Bishops Waltham PFS</td>\n      <td>PFS</td>\n      <td>SO32 1BA</td>\n      <td>50.95410</td>\n      <td>-1.21500</td>\n      <td>PO17 5</td>\n      <td>50.891426</td>\n      <td>-1.194287</td>\n      <td>7.333661</td>\n    </tr>\n    <tr>\n      <th>3916</th>\n      <td>1420</td>\n      <td>Bishops Waltham PFS</td>\n      <td>PFS</td>\n      <td>SO32 1BA</td>\n      <td>50.95410</td>\n      <td>-1.21500</td>\n      <td>SO32 2</td>\n      <td>50.934787</td>\n      <td>-1.217127</td>\n      <td>2.158665</td>\n    </tr>\n    <tr>\n      <th>10097</th>\n      <td>1420</td>\n      <td>Bishops Waltham PFS</td>\n      <td>PFS</td>\n      <td>SO32 1BA</td>\n      <td>50.95410</td>\n      <td>-1.21500</td>\n      <td>SO32 1</td>\n      <td>50.960326</td>\n      <td>-1.217575</td>\n      <td>0.748518</td>\n    </tr>\n    <tr>\n      <th>11912</th>\n      <td>1399</td>\n      <td>Aylesbury Gh Rd PFS</td>\n      <td>PFS</td>\n      <td>HP19 8PQ</td>\n      <td>51.82202</td>\n      <td>-0.82514</td>\n      <td>HP27 7</td>\n      <td>51.778221</td>\n      <td>-0.771607</td>\n      <td>7.685940</td>\n    </tr>\n    <tr>\n      <th>12623</th>\n      <td>1399</td>\n      <td>Aylesbury Gh Rd PFS</td>\n      <td>PFS</td>\n      <td>HP19 8PQ</td>\n      <td>51.82202</td>\n      <td>-0.82514</td>\n      <td>HP20 9</td>\n      <td>51.820160</td>\n      <td>-0.816227</td>\n      <td>1.011785</td>\n    </tr>\n  </tbody>\n</table>\n</div>"
     },
     "execution_count": 10,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "final_data[\"distance\"] = haversine_np(final_data.latitude,\n",
    "                                      final_data.longitude,\n",
    "                                      final_data.latitude_store,\n",
    "                                      final_data.longitude_store)\n",
    "within_8_km = final_data[final_data['distance']<8]\n",
    "within_8_km.head()\n"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%%\n"
    }
   }
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "outputs": [],
   "source": [
    "within_8_km.to_sql(name='pfs_postcode_within_8km',\n",
    "                   con=sf_con,\n",
    "                   index=False,\n",
    "                   if_exists='replace',\n",
    "                   chunksize=10000)"
   ],
   "metadata": {
    "collapsed": false,
    "pycharm": {
     "name": "#%%\n"
    }
   }
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 2
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython2",
   "version": "2.7.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 0
}

NameError: name 'true' is not defined